In [6]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



#from secret import *

### Firefox Driver

In [22]:
wd = webdriver.Firefox()
#driver.implicitly_wait(10)
wd.get('http://www.afl.com.au/stats')

In [69]:
def data_download (_year,_round):
    movable_part = "CD_S{year}014&roundId=CD_R{year}014{round}".format(year=_year, round= _round)
    #print(movable_part)
    script = '' + \
    'window.a = jQuery.ajax({'+\
    '        url:"http://www.afl.com.au/api/cfs/afl/statsCentre/teams?competitionId={}",'.format(movable_part) +\
    '        headers: {'+\
    '                    "X-media-mis-token":"bacd4d68e667f1bd2a5a8edea35a2a4f"'+\
    '                 }'+\
    '        });' 

    wd.execute_script(script)
    import time
    import json
    time.sleep(1)
    wd.execute_script("console.log('the dude', window.a.responseText);")
    result = wd.execute_script("return window.a.responseText")
    
    try:
        data = json.loads(result)
        filename= _year  + '_' + _round + '.txt'
        with open('afl_raw/' + filename, 'w') as f:
            f.write(result)
            print ('exported ' + filename)
        
    except Exception as e:
        filename= _year  + '_' + _round + '.txt'
        with open('afl_raw/' + filename, 'w') as f:
            f.write('error occured on writing')
            print ('export error on ' + filename)
        

            

    

In [41]:
year_list = ['2017','2016','2015','2014','2013','2012','2011','2010','2009','2008','2007','2006','2005','2004','2003','2002', '2001']
year_list_2 = ['2017','2016']
round_list = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27']

In [72]:
for i in year_list_2:
    current_year = i
    for i in round_list:
        current_round = i
        data_download(current_year,current_round)

exported 2017_01.txt
exported 2017_02.txt
exported 2017_03.txt
exported 2017_04.txt
exported 2017_05.txt
exported 2017_06.txt
exported 2017_07.txt
export error on 2017_08.txt
export error on 2017_09.txt
export error on 2017_10.txt
export error on 2017_11.txt
export error on 2017_12.txt
export error on 2017_13.txt
export error on 2017_14.txt
export error on 2017_15.txt
export error on 2017_16.txt
export error on 2017_17.txt
export error on 2017_18.txt
export error on 2017_19.txt
export error on 2017_20.txt
export error on 2017_21.txt
export error on 2017_22.txt
export error on 2017_23.txt
export error on 2017_24.txt
export error on 2017_25.txt
export error on 2017_26.txt
export error on 2017_27.txt
exported 2016_01.txt
exported 2016_02.txt
exported 2016_03.txt
exported 2016_04.txt
exported 2016_05.txt
exported 2016_06.txt
exported 2016_07.txt
exported 2016_08.txt
exported 2016_09.txt
exported 2016_10.txt
exported 2016_11.txt
exported 2016_12.txt
exported 2016_13.txt
exported 2016_14.txt

In [70]:
current_year = '2017'
current_round ='08'

In [71]:
data_download(current_year,current_round)

export error on 2017_08.txt


In [39]:
data['lists'][0]['team']

{'teamAbbr': 'GWS',
 'teamId': 'CD_T1010',
 'teamName': 'GWS Giants',
 'teamNickname': 'Giants'}

In [37]:
data['lists'][0]['stats']['totals']

{'behinds': 9.0,
 'bounces': 5.0,
 'clangers': 59.0,
 'clearances': {'centreClearances': 11.0,
  'stoppageClearances': 29.0,
  'totalClearances': 40.0},
 'contestedMarks': 15.0,
 'contestedPossessions': 154.0,
 'disposalEfficiency': 71.1,
 'disposals': 381.0,
 'dreamTeamPoints': 1638.0,
 'freesAgainst': 26.0,
 'freesFor': 18.0,
 'goalAccuracy': 52.4,
 'goalAssists': 8.0,
 'goalEfficiency': 22.0,
 'goals': 11.0,
 'handballs': 163.0,
 'hitouts': 62.0,
 'inside50s': 50.0,
 'intercepts': 63.0,
 'interchangeCounts': {'interchangeCap': 90.0,
  'interchangeCountQ1': 19.0,
  'interchangeCountQ2': 20.0,
  'interchangeCountQ3': 22.0,
  'interchangeCountQ4': 21.0,
  'totalInterchangeCount': 82.0},
 'kicks': 218.0,
 'lastUpdated': '2017-04-28T12:43:21.605+0000',
 'marks': 95.0,
 'marksInside50': 10.0,
 'metresGained': None,
 'onePercenters': 47.0,
 'ranking': None,
 'ratingPoints': None,
 'rebound50s': 53.0,
 'scoreInvolvements': None,
 'shotEfficiency': 42.0,
 'shotsAtGoal': 21.0,
 'superGoals': 

In [23]:
print (result)

None


In [16]:
select = Select(driver.find_element_by_id('selTeamSeason'))
#test = select.select_by_value('CD_S2013014')
#print (test)

In [ ]:
select = Select(b.find_element_by_id('selTeamSeason'))
select.select_by_value('CD_S2015014')


#https://sqa.stackexchange.com/questions/1355/what-is-the-correct-way-to-select-an-option-using-seleniums-python-webdriver

### Chrome Driver

#chromedriver = "Q:\\Users\\ssweeney\\Downloads\\chromedriver_win32\\chromedriver"
#os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get("http://google.com")
#driver.quit()

In [ ]:
soup = BeautifulSoup(req.text, 'html.parser')
stats_table = soup.find("div", { "class" : "stats-options" })



CD_S2017014

<select id="fruits01" class="select" name="fruits">
    <option value="0">Choose your fruits:</option>
    <option value="1">Banana</option>
    <option value="2">Mango</option>
</select>

In [ ]:
driver = webdriver.Firefox()
driver.get('http://www.afl.com.au/stats')

select = Select(driver.find_element_by_id('selTeamSeason'))

# select by visible text
select.select_by_visible_text('Banana')





# select by value 
select.select_by_value('1')




selTeamSeason
year_value = 'CD_S' + Year + '014'

selTeamMatch
month_value = 'CD_R' + Year + '014' + Round

selTeamMatch
match_value = 'CD_M' + Year + '014' + Round + Match


CD_M20170140201

extract all of that and run a loop that finds the input id that links to that

http://stackoverflow.com/questions/7867537/selenium-python-drop-down-menu-option-value
    
    selenium.click("id=myButton”)
    driver.find_element_by_id("checkbox_id").click()
    http://selenium-python.readthedocs.io/en/latest/locating-elements.html

In [ ]:
def get_season(link):
    driver = webdriver.Firefox()
    driver.get('url')
    Year_Selector = filters.find("select", { "id" : "selTeamSeason" })
    Years = Year_Selector.find_all('option')
    print (Years)
    #for year in Years:
        #print (year.string)

In [ ]:
get_season(Selector)

In [ ]:
def get_round(link):
    Round_Selector = filters.find("select", { "id" : "selTeamRound" })
    Round = Round_Selector.find_all('option')
    #print (Years)
    #for year in Years:
        #print (year.string)